In [1]:
# Install required packages in the folllowing few cells

In [2]:
%%capture
 # capture hides output
!pip install yfinance pandas numpy scikit-learn ta pandas_ta xgboost

In [3]:
%%capture
# hidden output
!pip install alpha_vantage

In [4]:
%%capture
# hidden output, !pip install is required to run just on a machine
!pip install requests pandas_datareader


#### IMPORTS

In [184]:
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from datetime import time
from datetime import timedelta
from pandas.tseries.offsets import BDay
import matplotlib.lines as mlines
from dotenv import load_dotenv
import os
import matplotlib.patches as patches
import seaborn as sns
import warnings
import requests
import json
import pandas_ta as ta
from ta import trend, volatility, momentum, volume
import yfinance as yf
import time
from sklearn.model_selection import train_test_split


from numpy import array
from importlib import reload # to reload modules if we made changes to them without restarting kernel


In [185]:
data_yf = pd.read_csv('data/all_stock_data.csv')

In [186]:
data_yf.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,macd,macd_signal,...,WILLR_14_lag_3,WILLR_14_lag_4,WILLR_14_lag_5,OBV_lag_1,OBV_lag_2,OBV_lag_3,OBV_lag_4,OBV_lag_5,gain_loss_pct,win
0,2005-01-01,-0.497876,-0.496862,-0.497650,-0.496733,-0.450456,36.113739,AAPL,-0.805879,0.583648,...,0.800159,0.954954,0.967344,-6.249355,-6.249623,-6.250280,-6.248141,-6.246777,1.443757,1
1,2005-01-08,-0.497349,-0.496377,-0.497575,-0.496634,-0.450371,75.058095,AAPL,-2.215210,-0.005819,...,0.970425,0.800166,0.954961,-6.298288,-6.249074,-6.249342,-6.250000,-6.247860,0.054783,1
2,2005-01-15,-0.497347,-0.496663,-0.497097,-0.496604,-0.450346,23.887793,AAPL,-3.294118,-0.707636,...,1.034995,0.970432,0.800173,-6.196910,-6.298005,-6.248793,-6.249062,-6.249719,0.139038,1
3,2005-01-22,-0.497229,-0.496422,-0.496797,-0.496242,-0.450036,25.660756,AAPL,-4.101576,-1.441405,...,-1.908937,1.035001,0.970439,-6.164441,-6.196632,-6.297723,-6.248513,-6.248781,0.860787,1
4,2005-01-29,-0.496854,-0.495821,-0.496372,-0.495736,-0.449605,31.220490,AAPL,-4.687041,-2.153361,...,-1.908791,-1.908927,1.035009,-6.129584,-6.164164,-6.196353,-6.297440,-6.248232,1.184667,1


In [187]:
data_yf.isna().sum()


Date             0
Open             0
High             0
Low              0
Close            0
                ..
OBV_lag_3        0
OBV_lag_4        0
OBV_lag_5        0
gain_loss_pct    0
win              0
Length: 128, dtype: int64

In [188]:
data_yf.isna().any().any()

False

In [189]:
data_yf

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,macd,macd_signal,...,WILLR_14_lag_3,WILLR_14_lag_4,WILLR_14_lag_5,OBV_lag_1,OBV_lag_2,OBV_lag_3,OBV_lag_4,OBV_lag_5,gain_loss_pct,win
0,2005-01-01,-0.497876,-0.496862,-0.497650,-0.496733,-0.450456,36.113739,AAPL,-0.805879,0.583648,...,0.800159,0.954954,0.967344,-6.249355,-6.249623,-6.250280,-6.248141,-6.246777,1.443757,1
1,2005-01-08,-0.497349,-0.496377,-0.497575,-0.496634,-0.450371,75.058095,AAPL,-2.215210,-0.005819,...,0.970425,0.800166,0.954961,-6.298288,-6.249074,-6.249342,-6.250000,-6.247860,0.054783,1
2,2005-01-15,-0.497347,-0.496663,-0.497097,-0.496604,-0.450346,23.887793,AAPL,-3.294118,-0.707636,...,1.034995,0.970432,0.800173,-6.196910,-6.298005,-6.248793,-6.249062,-6.249719,0.139038,1
3,2005-01-22,-0.497229,-0.496422,-0.496797,-0.496242,-0.450036,25.660756,AAPL,-4.101576,-1.441405,...,-1.908937,1.035001,0.970439,-6.164441,-6.196632,-6.297723,-6.248513,-6.248781,0.860787,1
4,2005-01-29,-0.496854,-0.495821,-0.496372,-0.495736,-0.449605,31.220490,AAPL,-4.687041,-2.153361,...,-1.908791,-1.908927,1.035009,-6.129584,-6.164164,-6.196353,-6.297440,-6.248232,1.184667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393423,2023-11-25,1.300387,1.248745,1.299685,1.283046,1.344140,-0.214561,BIO,-1.762781,-1.712488,...,-0.939853,-1.357087,-1.820190,1.223875,1.223835,1.223817,1.223761,1.223713,-0.200466,-1
393424,2023-12-02,1.277312,1.280437,1.313742,1.266341,1.327306,-0.214341,BIO,-1.719633,-1.736968,...,-1.046805,-0.939844,-1.357077,1.223863,1.223840,1.223800,1.223782,1.223726,-0.125076,-1
393425,2023-12-09,1.278071,1.335034,1.255892,1.277924,1.338978,-0.208835,BIO,-1.656665,-1.743115,...,-0.776454,-1.046796,-0.939834,1.223851,1.223828,1.223805,1.223764,1.223747,0.007847,0
393426,2023-12-16,1.303308,1.336508,1.331884,1.363023,1.424733,-0.212919,BIO,-1.518174,-1.718477,...,-0.872208,-0.776445,-1.046787,1.223871,1.223816,1.223793,1.223770,1.223729,0.732549,1


In [198]:
# Initialize a list to collect DataFrames
#symbols = list(data_yf['Ticker'].value_counts().index)[0:300] #eases the number

symbols = list(data_yf['Ticker'].value_counts().index)

df_list = []

# Loop over each unique ticker
for ticker in range(len(symbols)):
    df = data_yf[data_yf['Ticker'] == symbols[ticker]]
    
    # Rename all columns in df to "original_column_nameTicker"
    df.columns = [str(col) + str(symbols[ticker]) for col in df.columns]
    
    df = df.reset_index().drop(columns=['index'])#[0:990]
    
    # Append modified df to the list
    df_list.append(df)



In [199]:
num_null = 0

for df in df_list:
    if df.isna().any().any() == True:
        num_null +=1 
        
data = pd.concat(df_list, axis=1)


data.set_index('DateFITB', inplace=True)

In [200]:
date_values = [col for col in data.columns if 'Date' in col]
data.drop(columns=date_values, inplace=True)

In [201]:
data.isna().sum()

OpenHUBB            0
HighHUBB            0
LowHUBB             0
CloseHUBB           0
Adj CloseHUBB       0
                   ..
OBV_lag_3BIO        1
OBV_lag_4BIO        1
OBV_lag_5BIO        1
gain_loss_pctBIO    1
winBIO              1
Length: 48434, dtype: int64

In [202]:
data.shape

(992, 48434)

Now, we will do Predict then Optimize -- Shared Learning

<h2> Predict then Optimize -- Shared Learning</h2>

In [203]:
from sklearn.preprocessing import StandardScaler

float_columns = data.select_dtypes(include=['float64']).columns

# # Initialize the StandardScaler
#scaler = StandardScaler()

# # Standardize only the float columns
#data[float_columns] = scaler.fit_transform(data[float_columns])

In [212]:
data = data[float_columns]

data.dropna(inplace=True)
#data.fillna(-99, inplace=True) 


In [213]:
data.shape

(991, 46051)

In [214]:
data.head()

,OpenHUBB,HighHUBB,LowHUBB,CloseHUBB,Adj CloseHUBB,VolumeHUBB,macdHUBB,macd_signalHUBB,rsiHUBB,bb_highHUBB,...,WILLR_14_lag_3BIO,WILLR_14_lag_4BIO,WILLR_14_lag_5BIO,OBV_lag_1BIO,OBV_lag_2BIO,OBV_lag_3BIO,OBV_lag_4BIO,OBV_lag_5BIO,gain_loss_pctBIO,winBIO
DateFITB,,,,,,,,,,,,,,,,,,,,,
2005-01-01,-0.202025,-0.210007,-0.215114,-0.221158,-0.280157,-0.218504,0.015377,-0.015946,-0.815981,-0.195355,...,0.970353,0.270101,0.033455,1.223428,1.223374,1.223364,1.223261,1.223184,-0.880709,-1.0
2005-01-08,-0.220836,-0.216980,-0.213011,-0.209575,-0.272929,-0.215039,-0.000302,-0.012821,-0.540513,-0.195845,...,0.743140,0.970359,0.270110,1.223424,1.223393,1.223339,1.223329,1.223226,-0.107093,0.0
2005-01-15,-0.209210,-0.215053,-0.207004,-0.213999,-0.275689,-0.218238,-0.016235,-0.013721,-0.627708,-0.197624,...,0.781335,0.743147,0.970367,1.223427,1.223389,1.223358,1.223304,1.223294,0.510395,1.0
2005-01-22,-0.214234,-0.221856,-0.219319,-0.219586,-0.279176,-0.216781,-0.033149,-0.018051,-0.739635,-0.198124,...,-1.897025,0.781342,0.743154,1.223429,1.223392,1.223354,1.223323,1.223269,-0.403426,-1.0
2005-01-29,-0.218382,-0.214656,-0.210308,-0.207712,-0.271767,-0.218320,-0.036216,-0.022169,-0.435145,-0.198329,...,-1.853393,-1.897015,0.781350,1.223428,1.223394,1.223357,1.223319,1.223287,0.260174,1.0


In [215]:
import tensorflow as tf
from scipy.optimize import minimize

In [216]:
device_name = "GPU" if tf.config.experimental.list_physical_devices('GPU') else "cpu"
print("Using device:", device_name)

Using device: GPU


In [217]:
physical_devices = tf.config.list_physical_devices()
print("Available devices:", physical_devices)

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [230]:
# Prepare features and targets for model training
def prepare_features_targets(data_2):
#     indicators = calculate_technical_indicators(data)
#     returns = np.log(data / data.shift(1)).add_suffix('_returns')
#     full_data = pd.concat([indicators, returns], axis=1).dropna()
    features = data_2[[col for col in data_2.columns if 'gain' not in col]]
    targets = data_2[[col for col in data_2.columns if 'gain' in col]]
    return features, targets

# Build and compile a neural network
def build_model(input_shape, output_shape):
    with tf.device(device_name):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(output_shape)
        ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Portfolio optimization function with variance consideration and risk aversion
def optimize_portfolio(weights, model, features, historical_returns, risk_aversion=0.5):
    weights = weights / np.sum(weights)  # Normalize weights
    pred_returns = model.predict(np.array([features.iloc[-1]]))[0]
    expected_return = np.dot(weights, pred_returns)
    covariance_matrix = historical_returns.cov()
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    # Objective function with risk aversion parameter
    return -expected_return + risk_aversion * portfolio_variance  # Maximize returns and penalize variance


In [219]:
#set index to datetime
data.index = pd.to_datetime(data.index)

In [222]:
# Main script
#features, targets = prepare_features_targets(data)
with tf.device(device_name):
    print(device_name)
    # Your model code here
    split_date = pd.Timestamp('2022-01-01')
    # Prepare your features and targets first
    features, targets = prepare_features_targets(data)
    # Split the data based on the threshold date
    train_features = features[features.index < split_date]
    test_features = features[features.index >= split_date]
    train_targets = targets[targets.index < split_date]
    test_targets = targets[targets.index >= split_date]
    #features, targets = prepare_features_targets(data)
    #train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)
    model = build_model(train_features.shape[1], train_targets.shape[1])
    model.fit(train_features, train_targets, epochs=3, batch_size=5)


# train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)

# model = build_model(train_features.shape[1], train_targets.shape[1])
# model.fit(train_features, train_targets, epochs=3, batch_size=5)

constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
initial_weights = np.ones(len(symbols)) / len(symbols)  # Adjust this based on the actual number of features or stocks.
#initial_weights = kernel_initializer="he_normal"
bounds = tuple((0, 1) for _ in symbols)


# optimized_result = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.999999999999999999999999), bounds=bounds, constraints=constraints)
# optimized_weights = optimized_result.x

with tf.device(device_name):
    # Optimizing the portfolio on training data
    train_optimized_result = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.8), bounds=bounds, constraints=constraints)
    optimized_weights_train = train_optimized_result.x

    # Optimizing the portfolio on testing data
    test_optimized_result = minimize(optimize_portfolio, initial_weights, args=(model, test_features, test_targets, 0.8), bounds=bounds, constraints=constraints)
    optimized_weights_test = test_optimized_result.x

    # Calculating ROI
    train_predicted_returns = model.predict(train_features)
    train_roi = np.dot(optimized_weights_train, train_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

    test_predicted_returns = model.predict(test_features)
    test_roi = np.dot(optimized_weights_test, test_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment


GPU
Epoch 1/3
178/178 [==============================] - 2s 10ms/step - loss: 53.7994
Epoch 2/3
178/178 [==============================] - 1s 8ms/step - loss: 21.2811
Epoch 3/3
4/4 [==============================] - 0s 70ms/step


In [223]:
print("Optimized Portfolio Weights (Train):", optimized_weights_train)
print("Training ROI:", train_roi)
print("Optimized Portfolio Weights (Test):", optimized_weights_test)
print("Testing ROI:", test_roi)

Optimized Portfolio Weights (Train): [0.00000000e+00 1.29690812e-11 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 6.27154520e-01 3.39442047e-11
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 6.03384914e-12 3.37675237e-11 6.20712154e-12 2.80115160e-11
 0.00000000e+00 8.01713308e-12 1.91911994e-11 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 7.11321731e-12
 0.00000000e+00 8.98656714e-12 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 3.72845481e-01 8.54890285e-12 1.72645278e-11
 1.38445739e-11 1.60744889e-11 1.47267031e-11 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 9.77941947e-12
 0.00000000e+00 0.00000000e+00 7.09060334e-12 5.83812647e-12
 0.00000000e+00 2.42323485e-11 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.67470257e-11 1.74021282e-11 2.23625353e-11
 1.74453559e-11 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 4.14764237e-12 2.24092347e-12 4.

In [225]:
def print_non_zero_weights(weights, title):
    # Using a threshold to consider weights effectively zero
    threshold = 1e-10
    non_zero_weights = {index: weight for index, weight in enumerate(weights) if abs(weight) > threshold}
    print(f"{title} (non-zero weights):")
    for index, weight in non_zero_weights.items():
        print(f"Index {index}: Weight {weight}")

# Assuming optimized_weights_train and optimized_weights_test are your weight arrays
print_non_zero_weights(optimized_weights_train, "Optimized Portfolio Weights (Train)")
print("Training ROI:", train_roi)
print_non_zero_weights(optimized_weights_test, "Optimized Portfolio Weights (Test)")
print("Testing ROI:", test_roi)


Optimized Portfolio Weights (Train) (non-zero weights):
Index 6: Weight 0.6271545204979063
Index 33: Weight 0.3728454805511894
Training ROI: 11854.76937029871
Optimized Portfolio Weights (Test) (non-zero weights):
Index 87: Weight 0.14974000791786696
Index 220: Weight 0.8502599927612026
Testing ROI: 52440.88135448226


In [226]:

print("Training ROI:", train_roi)

print("Testing ROI:", test_roi)

Training ROI: 11854.76937029871
Testing ROI: 52440.88135448226


In [227]:
max(optimized_weights_train)

0.6271545204979063

In [228]:
train_predicted_returns

array([[ -4.9756317 ,  10.076855  , -10.775794  , ...,  -1.243499  ,
         25.664215  ,   0.22510588],
       [-11.339482  ,   9.285493  ,  -9.809999  , ...,  -4.419855  ,
         25.63416   ,  13.526826  ],
       [-15.376298  ,  13.459332  ,   1.3245592 , ...,  16.026968  ,
         17.245628  ,   2.0404477 ],
       ...,
       [ -2.5298922 ,   3.075103  ,  -1.6328329 , ...,   4.6996703 ,
         -1.3780109 ,  -3.2350898 ],
       [ -1.9434698 ,   3.5193384 ,  -3.501445  , ...,   3.4300802 ,
         -1.9215908 ,  -1.9122102 ],
       [ -1.4149852 ,   2.8484564 ,  -4.296486  , ...,   3.4999907 ,
         -1.8777404 ,  -1.186506  ]], dtype=float32)

In [229]:
# Assuming 'data' is indexed by datetime
yearly_splits = pd.date_range(start='2000-01-01', end='2024-01-01', freq='Y')

annual_weights = {}
annual_returns = {}

for start, end in zip(yearly_splits[:-1], yearly_splits[1:]):
    year_data = data[start:end]
    features, targets = prepare_features_targets(year_data)
    
    # Split into training and testing data if necessary or use the full year for training
    train_features, train_targets = features, targets
    
    # Initialize and train model
    model = build_model(train_features.shape[1], train_targets.shape[1])
    model.fit(train_features, train_targets, epochs=3, batch_size=5)
    
    # Optimize portfolio
    result = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.8), bounds=bounds, constraints=constraints)
    annual_weights[start.year] = result.x
    
    # Simulate or calculate the ROI for the year
    predicted_returns = model.predict(train_features)
    annual_returns[start.year] = np.dot(result.x, predicted_returns.mean(axis=0)) * 10000  # Modify as per actual investment

# Display the results
print("Annual Weights:", annual_weights)
print("Annual Returns:", annual_returns)


ValueError: Expected input data to be non-empty.

In [ ]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from scipy.optimize import minimize
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# # Prepare features and targets for model training
# def prepare_features_targets(data):
#     features = data[[col for col in data.columns if 'gain' not in col]]
#     targets = data[[col for col in data.columns if 'gain' in col]]
#     return features, targets

# # Build and compile a neural network
# def build_model(input_shape, output_shape):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
#         tf.keras.layers.Dense(32, activation='relu'),
#         tf.keras.layers.Dense(output_shape)
#     ])
#     model.compile(optimizer='adam', loss='mse')
#     return model

# # Portfolio optimization function with variance consideration and risk aversion
# def optimize_portfolio(weights, model, features, historical_returns, risk_aversion=0.8):
#     weights = weights / np.sum(weights)  # Normalize weights
#     pred_returns = model.predict(features)[-1]  # Predicting the last set of features
#     expected_return = np.dot(weights, pred_returns)
#     covariance_matrix = historical_returns.cov()
#     portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
#     return -expected_return + risk_aversion * portfolio_variance  # Maximize returns and penalize variance

# column_values = [col for col in data.columns if 'EXC' or 'APPL' in col]
# data_2 = data[column_values]

# symbols = ['EXC', 'AAPL']

# # Prepare features and targets
# features, targets = prepare_features_targets(data_2)

# # Split data into training and testing sets
# train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)

# # Build model
# model = build_model(train_features.shape[1], train_targets.shape[1])

# # Train model
# model.fit(train_features, train_targets, epochs=3, batch_size=32)

# # Optimizing portfolio using training data
# # train_returns = train_features.pct_change().dropna()
# initial_weights = np.ones(len(symbols)) / len(symbols)
# bounds = tuple((0, 1) for _ in symbols)
# constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}

# optimized_result_train = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.8), bounds=bounds, constraints=constraints)
# optimized_weights_train = optimized_result_train.x

# # Calculate ROI for the training set
# train_predicted_returns = model.predict(train_features)
# train_roi = np.dot(optimized_weights_train, train_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

# # Optimizing portfolio using testing data
# test_returns = test_features.pct_change().dropna()
# optimized_result_test = minimize(optimize_portfolio, initial_weights, args=(model, test_features, test_targets, 0.8), bounds=bounds, constraints=constraints)
# optimized_weights_test = optimized_result_test.x

# # Calculate ROI for the testing set
# test_predicted_returns = model.predict(test_features)
# test_roi = np.dot(optimized_weights_test, test_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

# print(f"Training ROI: ${train_roi:.2f}")
# print(f"Testing ROI: ${test_roi:.2f}")
# print("Optimized Weights for Training Set:", optimized_weights_train)
# print("Optimized Weights for Testing Set:", optimized_weights_test)


Non Shared Learning


In [ ]:
data_yf

In [ ]:
#Need a train loader and a test loader
#Need to convert to optDataset, then wrap with DataLoader

In [ ]:
!pip install PyPl gurobipy pyepo

Create a LSTM-based model for stock prediction tasks

In [ ]:
import torch.nn.functional as F
from torch import nn
import torch 
class StockLSTM(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        super(StockLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        all_h, (h, c) = self.lstm(x)
        out = self.fc(all_h) # Apply Linear layer to outputs from all the hidden state.
        return out

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pyepo
from pyepo.model.grb import optGrbModel
import torch
from torch import nn
from torch.utils.data import DataLoader

m = 50 # change based on number of assets
cov = np.cov(np.random.randn(10, m), rowvar=False) # covariance matrix
optmodel = pyepo.model.grb.portfolioModel(m, cov) # build model

Auto-Sklearn cannot be imported.
Restricted license - for non-production use only - expires 2025-11-24


In [ ]:
import time

# train model
def trainModel(reg, loss_func, method_name, num_epochs=20, lr=1e-2):
    # set adam optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=lr)
    # train mode
    reg.train()
    # init log
    train_loss_log = []
    loss_log_regret = [pyepo.metric.regret(reg, optmodel, loader_test)]
    # init elpased time
    elapsed = 0
    for epoch in range(num_epochs):
        # start timing
        tick = time.time()
        # load data
        train_loss = 0
        for i, data in enumerate(loader_train):
            x, c, w, z = data
            # cuda
            if torch.cuda.is_available():
                x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
            # forward pass
            cp = reg(x)
            if method_name == "spo+":
                loss = loss_func(cp, c, w, z)
            elif method_name == "mse":
                loss = loss_func(cp, c)
            else:
                raise ValueError("Method name {} not supported".format(method_name))
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # record time
            tock = time.time()
            elapsed += tock - tick
            train_loss += loss.item()
        train_loss /= len(loader_train)
        train_loss_log.append(train_loss)
        regret = pyepo.metric.regret(reg, optmodel, loader_test)
        loss_log_regret.append(regret)
        print("Epoch {:2},  Loss: {:9.4f},  Regret: {:7.4f}%".format(epoch+1, train_loss, regret*100))
    print("Total Elapsed Time: {:.2f} Sec.".format(elapsed))
    return train_loss_log, loss_log_regret

Create a Predict-then-Optimize Model

In [ ]:

spop = pyepo.func.SPOPlus(optmodel, processes=1)

In [ ]:
# Hyperparameters
VOCAB_SIZE = #fill in once data is imported
EMBEDDING_DIM = #param we can optimize
HIDDEN_DIM = #also a param to optimize
learning_rate = #another optimizable param
epoch = #optimize
num_layers = #optimize
epochs = 20
learning_rate = 2e-3
method_name = "spo+"

# Instantiate the model
lstm = StockLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM,num_layers=num_layers)
loss_log_lstm_spo, loss_log_regret_lstm_spo = trainModel(lstm, loss_func=spop, method_name=method_name)
